In [2]:
# =========================================================
# 1. Importação de bibliotecas
# =========================================================

import requests   # Para fazer requisições HTTP
import json       # Para trabalhar com respostas em formato JSON
import pandas as pd  # Para manipulação e análise de dados tabulares


In [3]:
# =========================================================
# 2. Função para coletar dados do CPI
# =========================================================
def get_cpi_data(start_year, end_year, series_ids=["CUUR0000SA0"]):
    """
    Obtém dados do CPI (Consumer Price Index) usando a API do BLS (Bureau of Labor Statistics).

    Args:
        start_year (int): Ano inicial da coleta.
        end_year (int): Ano final da coleta.
        series_ids (list): Lista de IDs das séries de dados.
                           Exemplo: ["CUUR0000SA0"] para CPI-U (All items).

    Returns:
        DataFrame: Dados no formato tabular com colunas
                   (series_id, year, month, value).
    """

    # Define cabeçalho da requisição
    headers = {'Content-type': 'application/json'}

    # Define corpo da requisição (anos + séries)
    data = json.dumps({
        "seriesid": series_ids,
        "startyear": str(start_year),
        "endyear": str(end_year)
    })

    # Faz requisição POST à API do BLS
    p = requests.post(
        'https://api.bls.gov/publicAPI/v2/timeseries/data/',
        data=data, headers=headers
    )
    json_data = json.loads(p.text)

    # Caso a requisição seja bem-sucedida
    if json_data['status'] == 'REQUEST_SUCCEEDED':
        all_data = []

        # Percorre todas as séries retornadas
        for series in json_data['Results']['series']:
            sid = series['seriesID']
            for item in series['data']:
                year = int(item['year'])
                period = item['period']  # formato "M01", "M02", ...
                value = float(item['value'])

                # Considera apenas períodos mensais (ignora agregados anuais)
                if period.startswith('M'):
                    month = int(period[1:])
                    all_data.append({
                        'series_id': sid,
                        'year': year,
                        'month': month,
                        'value': value
                    })

        # Retorna DataFrame organizado
        df = pd.DataFrame(all_data)
        return df.sort_values(
            by=['series_id', 'year', 'month']
        ).reset_index(drop=True)

    else:
        # Caso a API retorne erro
        print('Erro ao obter dados do BLS')
        return None


In [4]:
# =========================================================
# 3. Exemplo de uso
# =========================================================

df = get_cpi_data(2024, 2025, series_ids=["CUUR0000SA0"])

# Reorganiza os dados em formato "wide"
df_pivot = df.pivot(
    index=["year", "month"],
    columns="series_id",
    values="value"
).reset_index()

# Renomeia colunas para facilitar leitura
df_pivot = df_pivot.rename(columns={
    "CUUR0000SA0": "All items"
})

# Exibe os dados no console
print(df_pivot)

# Opcional: salva em CSV
df_pivot.to_csv("cpi_data.csv", index=False)
print("\nArquivo 'cpi_data.csv' salvo com sucesso!")

series_id  year  month  All items
0          2024      1    308.417
1          2024      2    310.326
2          2024      3    312.332
3          2024      4    313.548
4          2024      5    314.069
5          2024      6    314.175
6          2024      7    314.540
7          2024      8    314.796
8          2024      9    315.301
9          2024     10    315.664
10         2024     11    315.493
11         2024     12    315.605
12         2025      1    317.671
13         2025      2    319.082
14         2025      3    319.799
15         2025      4    320.795
16         2025      5    321.465
17         2025      6    322.561
18         2025      7    323.048

Arquivo 'cpi_data.csv' salvo com sucesso!
